**If you are using Google Colab' please run the following cell**

In [ ]:
#!rm -r DPPy
#!git clone https://github.com/guilgautier/DPPy.git
#!pip install scipy --upgrade
#!pip install DPPy/.

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#DPPy-tutorial" data-toc-modified-id="DPPy-tutorial-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://github.com/guilgautier/DPPy" target="_blank">DPPy</a> tutorial</a></div><div class="lev1 toc-item"><a href="#I.-Check-installation" data-toc-modified-id="I.-Check-installation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>I. Check installation</a></div><div class="lev1 toc-item"><a href="#💣-To-interact-with-source-code,-please-uncomment-the-following-cell" data-toc-modified-id="💣-To-interact-with-source-code,-please-uncomment-the-following-cell-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>💣 To interact with source code, please uncomment the following cell</a></div><div class="lev1 toc-item"><a href="#II.-Sample-DPPs-with-DPPy" data-toc-modified-id="II.-Sample-DPPs-with-DPPy-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>II. Sample DPPs with DPPy</a></div><div class="lev2 toc-item"><a href="#II.-1.-Continuous-DPPs:-$\beta$-Ensembles" data-toc-modified-id="II.-1.-Continuous-DPPs:-$\beta$-Ensembles-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>II. 1. Continuous DPPs: <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-1-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 0.646em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.532em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.517em, 1000.53em, 2.616em, -999.998em); top: -2.309em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.04em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.313em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.27em; border-left: 0px solid; width: 0px; height: 1.093em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi></math></span></span><script type="math/tex" id="MathJax-Element-1">\beta</script>-Ensembles</a></div><div class="lev3 toc-item"><a href="#See-documentation" data-toc-modified-id="See-documentation-411"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>See <a href="https://dppy.readthedocs.io/en/latest/continuous_dpps/index.html" target="_blank">documentation</a></a></div><div class="lev3 toc-item"><a href="#d.-Circular" data-toc-modified-id="d.-Circular-412"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>d. Circular</a></div><div class="lev4 toc-item"><a href="#Visualization-of-the-cristallization-as-$\beta\in-\mathbb{N}^*$-increases" data-toc-modified-id="Visualization-of-the-cristallization-as-$\beta\in-\mathbb{N}^*$-increases-4121"><span class="toc-item-num">4.1.2.1&nbsp;&nbsp;</span>Visualization of the cristallization as <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-4-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><msup><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi mathvariant=&quot;double-struck&quot;>N</mi></mrow><mo>&amp;#x2217;</mo></msup></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-66" style="width: 3.515em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1002.92em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-67"><span class="mi" id="MathJax-Span-68" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-69" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-70" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-71"><span class="mrow" id="MathJax-Span-72"><span class="mi" id="MathJax-Span-73" style="font-family: STIXMathJax_DoubleStruck;">ℕ</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-74" style="font-size: 70.7%; font-family: STIXMathJax_Main;">∗</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>∗</mo></msup></math></span></span><script type="math/tex" id="MathJax-Element-4">\beta\in \mathbb{N}^*</script> increases</a></div><div class="lev4 toc-item"><a href="#Sample-using-full-matrix-model,-$\beta-\in-\{0,-1,-2,-4\}$" data-toc-modified-id="Sample-using-full-matrix-model,-$\beta-\in-\{0,-1,-2,-4\}$-4122"><span class="toc-item-num">4.1.2.2&nbsp;&nbsp;</span>Sample using full matrix model, <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-5-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>}</mo></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-75" style="width: 7.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1005.9em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-76"><span class="mi" id="MathJax-Span-77" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-78" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-79" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-80" style="font-family: STIXMathJax_Main;">0</span><span class="mo" id="MathJax-Span-81" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-82" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-83" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-84" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-85" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-86" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">4</span><span class="mo" id="MathJax-Span-87" style="font-family: STIXMathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script type="math/tex" id="MathJax-Element-5">\beta \in \{0, 1, 2, 4\}</script></a></div><div class="lev4 toc-item"><a href="#Sample-using-banded-(quindiagonal)-matrix-model,-$\beta-\in-\mathbb{N}^*$" data-toc-modified-id="Sample-using-banded-(quindiagonal)-matrix-model,-$\beta-\in-\mathbb{N}^*$-4123"><span class="toc-item-num">4.1.2.3&nbsp;&nbsp;</span>Sample using banded (quindiagonal) matrix model, <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-6-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><msup><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi mathvariant=&quot;double-struck&quot;>N</mi></mrow><mo>&amp;#x2217;</mo></msup></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-88" style="width: 3.515em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1002.92em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-89"><span class="mi" id="MathJax-Span-90" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-91" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-92" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.134em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.66em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-93"><span class="mrow" id="MathJax-Span-94"><span class="mi" id="MathJax-Span-95" style="font-family: STIXMathJax_DoubleStruck;">ℕ</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-96" style="font-size: 70.7%; font-family: STIXMathJax_Main;">∗</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>∗</mo></msup></math></span></span><script type="math/tex" id="MathJax-Element-6">\beta \in \mathbb{N}^*</script></a></div><div class="lev3 toc-item"><a href="#a.-Hermite-$\mu-=-\mathcal{N}$" data-toc-modified-id="a.-Hermite-$\mu-=-\mathcal{N}$-413"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>a. Hermite <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-8-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03BC;</mi><mo>=</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi class=&quot;MJX-tex-caligraphic&quot; mathvariant=&quot;script&quot;>N</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-102" style="width: 3.567em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.965em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.576em, 1002.97em, 2.826em, -999.998em); top: -2.498em; left: 0em;"><span class="mrow" id="MathJax-Span-103"><span class="mi" id="MathJax-Span-104" style="font-family: STIXMathJax_Normal-italic;">𝜇</span><span class="mo" id="MathJax-Span-105" style="font-family: STIXMathJax_Main; padding-left: 0.326em;">=</span><span class="texatom" id="MathJax-Span-106" style="padding-left: 0.326em;"><span class="mrow" id="MathJax-Span-107"><span class="mi" id="MathJax-Span-108" style="font-family: STIXMathJax_Variants-italic;"><span style="display: inline-block; overflow: hidden; height: 1px; width: 0.095em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.502em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.275em; border-left: 0px solid; width: 0px; height: 1.281em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>μ</mi><mo>=</mo><mrow class="MJX-TeXAtom-ORD"><mi class="MJX-tex-caligraphic" mathvariant="script">N</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-8">\mu = \mathcal{N}</script></a></div><div class="lev4 toc-item"><a href="#Sample-using-full-matrix-model,-$\beta-\in-\{0,-1,-2,-4\}$" data-toc-modified-id="Sample-using-full-matrix-model,-$\beta-\in-\{0,-1,-2,-4\}$-4131"><span class="toc-item-num">4.1.3.1&nbsp;&nbsp;</span>Sample using full matrix model, <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-9-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>}</mo></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-109" style="width: 7.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1005.9em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-110"><span class="mi" id="MathJax-Span-111" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-112" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-113" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-114" style="font-family: STIXMathJax_Main;">0</span><span class="mo" id="MathJax-Span-115" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-116" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-117" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-118" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-119" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-120" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">4</span><span class="mo" id="MathJax-Span-121" style="font-family: STIXMathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script type="math/tex" id="MathJax-Element-9">\beta \in \{0, 1, 2, 4\}</script></a></div><div class="lev4 toc-item"><a href="#Sample-using-banded-(tridiagonal)-matrix-model-$\beta-\in-\mathbb{R}_+$" data-toc-modified-id="Sample-using-banded-(tridiagonal)-matrix-model-$\beta-\in-\mathbb{R}_+$-4132"><span class="toc-item-num">4.1.3.2&nbsp;&nbsp;</span>Sample using banded (tridiagonal) matrix model <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-10-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi mathvariant=&quot;double-struck&quot;>R</mi></mrow><mo>+</mo></msub></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-122" style="width: 3.753em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.098em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1003.1em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-123"><span class="mi" id="MathJax-Span-124" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-125" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-126" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.313em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-127"><span class="mrow" id="MathJax-Span-128"><span class="mi" id="MathJax-Span-129" style="font-family: STIXMathJax_DoubleStruck;">ℝ</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="mo" id="MathJax-Span-130" style="font-size: 70.7%; font-family: STIXMathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><msub><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msub></math></span></span><script type="math/tex" id="MathJax-Element-10">\beta \in \mathbb{R}_+</script></a></div><div class="lev3 toc-item"><a href="#b.-Laguerre-$\mu-=-\mathcal{\Gamma}$" data-toc-modified-id="b.-Laguerre-$\mu-=-\mathcal{\Gamma}$-414"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>b. Laguerre <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-12-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03BC;</mi><mo>=</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi class=&quot;MJX-tex-caligraphic&quot; mathvariant=&quot;script&quot;>&amp;#x0393;</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-136" style="width: 3.15em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.595em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.715em, 1002.6em, 2.826em, -999.998em); top: -2.498em; left: 0em;"><span class="mrow" id="MathJax-Span-137"><span class="mi" id="MathJax-Span-138" style="font-family: STIXMathJax_Normal-italic;">𝜇</span><span class="mo" id="MathJax-Span-139" style="font-family: STIXMathJax_Main; padding-left: 0.326em;">=</span><span class="texatom" id="MathJax-Span-140" style="padding-left: 0.326em;"><span class="mrow" id="MathJax-Span-141"><span class="mi" id="MathJax-Span-142" style="font-family: STIXMathJax_Main-italic;">Γ<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.049em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.502em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.275em; border-left: 0px solid; width: 0px; height: 1.114em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>μ</mi><mo>=</mo><mrow class="MJX-TeXAtom-ORD"><mi class="MJX-tex-caligraphic" mathvariant="script">Γ</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-12">\mu = \mathcal{\Gamma}</script></a></div><div class="lev4 toc-item"><a href="#Sample-using-full-matrix-model,-$\beta-\in-\{0,-1,-2,-4\}$" data-toc-modified-id="Sample-using-full-matrix-model,-$\beta-\in-\{0,-1,-2,-4\}$-4141"><span class="toc-item-num">4.1.4.1&nbsp;&nbsp;</span>Sample using full matrix model, <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-13-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>}</mo></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-143" style="width: 7.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1005.9em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-144"><span class="mi" id="MathJax-Span-145" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-146" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-147" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-148" style="font-family: STIXMathJax_Main;">0</span><span class="mo" id="MathJax-Span-149" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-150" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-151" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-152" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-153" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-154" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">4</span><span class="mo" id="MathJax-Span-155" style="font-family: STIXMathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script type="math/tex" id="MathJax-Element-13">\beta \in \{0, 1, 2, 4\}</script></a></div><div class="lev4 toc-item"><a href="#Sample-using-banded-(tridiagonal)-matrix-model,-$\beta-\in-\mathbb{R}_+$" data-toc-modified-id="Sample-using-banded-(tridiagonal)-matrix-model,-$\beta-\in-\mathbb{R}_+$-4142"><span class="toc-item-num">4.1.4.2&nbsp;&nbsp;</span>Sample using banded (tridiagonal) matrix model, <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-14-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi mathvariant=&quot;double-struck&quot;>R</mi></mrow><mo>+</mo></msub></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-156" style="width: 3.753em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.098em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1003.1em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-157"><span class="mi" id="MathJax-Span-158" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-159" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-160" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.313em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-161"><span class="mrow" id="MathJax-Span-162"><span class="mi" id="MathJax-Span-163" style="font-family: STIXMathJax_DoubleStruck;">ℝ</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="mo" id="MathJax-Span-164" style="font-size: 70.7%; font-family: STIXMathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><msub><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msub></math></span></span><script type="math/tex" id="MathJax-Element-14">\beta \in \mathbb{R}_+</script></a></div><div class="lev3 toc-item"><a href="#c.-Jacobi-$\mu-=-\operatorname{Beta}$" data-toc-modified-id="c.-Jacobi-$\mu-=-\operatorname{Beta}$-415"><span class="toc-item-num">4.1.5&nbsp;&nbsp;</span>c. Jacobi <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-17-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03BC;</mi><mo>=</mo><mi>Beta</mi></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-178" style="width: 4.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.752em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.715em, 1003.75em, 2.826em, -999.998em); top: -2.498em; left: 0em;"><span class="mrow" id="MathJax-Span-179"><span class="mi" id="MathJax-Span-180" style="font-family: STIXMathJax_Normal-italic;">𝜇</span><span class="mo" id="MathJax-Span-181" style="font-family: STIXMathJax_Main; padding-left: 0.326em;">=</span><span class="mi" id="MathJax-Span-182" style="font-family: STIXMathJax_Main; padding-left: 0.326em;">Beta</span></span><span style="display: inline-block; width: 0px; height: 2.502em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.275em; border-left: 0px solid; width: 0px; height: 1.114em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>μ</mi><mo>=</mo><mi>Beta</mi></math></span></span><script type="math/tex" id="MathJax-Element-17">\mu = \operatorname{Beta}</script></a></div><div class="lev4 toc-item"><a href="#Sample-using-full-matrix-model,--$\beta-\in-\{0,-1,-2,-4\}$" data-toc-modified-id="Sample-using-full-matrix-model,--$\beta-\in-\{0,-1,-2,-4\}$-4151"><span class="toc-item-num">4.1.5.1&nbsp;&nbsp;</span>Sample using full matrix model,  <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-18-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>}</mo></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-183" style="width: 7.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1005.9em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-184"><span class="mi" id="MathJax-Span-185" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-186" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-187" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-188" style="font-family: STIXMathJax_Main;">0</span><span class="mo" id="MathJax-Span-189" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-190" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-191" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-192" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-193" style="font-family: STIXMathJax_Main;">,</span><span class="mn" id="MathJax-Span-194" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">4</span><span class="mo" id="MathJax-Span-195" style="font-family: STIXMathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>4</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script type="math/tex" id="MathJax-Element-18">\beta \in \{0, 1, 2, 4\}</script></a></div><div class="lev4 toc-item"><a href="#Sample-using-banded-(tridiagonal)-matrix-model,--$\beta-\in-\mathbb{R}_+$" data-toc-modified-id="Sample-using-banded-(tridiagonal)-matrix-model,--$\beta-\in-\mathbb{R}_+$-4152"><span class="toc-item-num">4.1.5.2&nbsp;&nbsp;</span>Sample using banded (tridiagonal) matrix model,  <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-19-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>&amp;#x2208;</mo><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi mathvariant=&quot;double-struck&quot;>R</mi></mrow><mo>+</mo></msub></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-196" style="width: 3.753em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.098em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1003.1em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-197"><span class="mi" id="MathJax-Span-198" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-199" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-200" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.313em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-201"><span class="mrow" id="MathJax-Span-202"><span class="mi" id="MathJax-Span-203" style="font-family: STIXMathJax_DoubleStruck;">ℝ</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.777em;"><span class="mo" id="MathJax-Span-204" style="font-size: 70.7%; font-family: STIXMathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>∈</mo><msub><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msub></math></span></span><script type="math/tex" id="MathJax-Element-19">\beta \in \mathbb{R}_+</script></a></div><div class="lev3 toc-item"><a href="#(e)-Ginibre-$\beta-=-2$-only" data-toc-modified-id="(e)-Ginibre-$\beta-=-2$-only-416"><span class="toc-item-num">4.1.6&nbsp;&nbsp;</span>(e) Ginibre <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-22-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B2;</mi><mo>=</mo><mn>2</mn></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-219" style="width: 2.826em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.363em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.669em, 1002.32em, 2.826em, -999.998em); top: -2.498em; left: 0em;"><span class="mrow" id="MathJax-Span-220"><span class="mi" id="MathJax-Span-221" style="font-family: STIXMathJax_Normal-italic;">𝛽<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.049em;"></span></span><span class="mo" id="MathJax-Span-222" style="font-family: STIXMathJax_Main; padding-left: 0.326em;">=</span><span class="mn" id="MathJax-Span-223" style="font-family: STIXMathJax_Main; padding-left: 0.326em;">2</span></span><span style="display: inline-block; width: 0px; height: 2.502em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.275em; border-left: 0px solid; width: 0px; height: 1.169em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>β</mi><mo>=</mo><mn>2</mn></math></span></span><script type="math/tex" id="MathJax-Element-22">\beta = 2</script> only</a></div><div class="lev2 toc-item"><a href="#II.-2.-Discrete-DPPs" data-toc-modified-id="II.-2.-Discrete-DPPs-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>II. 2. Discrete DPPs</a></div><div class="lev3 toc-item"><a href="#See-documentation" data-toc-modified-id="See-documentation-421"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>See <a href="https://dppy.readthedocs.io/en/latest/finite_dpps/index.html" target="_blank">documentation</a></a></div><div class="lev3 toc-item"><a href="#Build-inclusion-kernel-$\mathbf{K}$" data-toc-modified-id="Build-inclusion-kernel-$\mathbf{K}$-422"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Build inclusion kernel <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-23-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi mathvariant=&quot;bold&quot;>K</mi></mrow></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-224" style="width: 0.975em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.789em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.808em, 1000.79em, 2.78em, -999.998em); top: -2.637em; left: 0em;"><span class="mrow" id="MathJax-Span-225"><span class="texatom" id="MathJax-Span-226"><span class="mrow" id="MathJax-Span-227"><span class="mi" id="MathJax-Span-228" style="font-family: STIXMathJax_Normal-bold;">𝐊</span></span></span></span><span style="display: inline-block; width: 0px; height: 2.641em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.053em; border-left: 0px solid; width: 0px; height: 0.947em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="bold">K</mi></mrow></math></span></span><script type="math/tex" id="MathJax-Element-23">\mathbf{K}</script></a></div><div class="lev3 toc-item"><a href="#Declare-a-finite-DPP-in-DPPy" data-toc-modified-id="Declare-a-finite-DPP-in-DPPy-423"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Declare a finite DPP in DPPy</a></div><div class="lev4 toc-item"><a href="#1.-Via-eigen-decomposition" data-toc-modified-id="1.-Via-eigen-decomposition-4231"><span class="toc-item-num">4.2.3.1&nbsp;&nbsp;</span>1. Via eigen-decomposition</a></div><div class="lev4 toc-item"><a href="#2.-Via-its-kernel" data-toc-modified-id="2.-Via-its-kernel-4232"><span class="toc-item-num">4.2.3.2&nbsp;&nbsp;</span>2. Via its kernel</a></div><div class="lev3 toc-item"><a href="#Other-features" data-toc-modified-id="Other-features-424"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Other features</a></div><div class="lev4 toc-item"><a href="#a.-Compute-the-other-kernel" data-toc-modified-id="a.-Compute-the-other-kernel-4241"><span class="toc-item-num">4.2.4.1&nbsp;&nbsp;</span>a. Compute the <em>other</em> kernel</a></div><div class="lev4 toc-item"><a href="#b.-Compute/plot-the-underlying-kernel" data-toc-modified-id="b.-Compute/plot-the-underlying-kernel-4242"><span class="toc-item-num">4.2.4.2&nbsp;&nbsp;</span>b. Compute/plot the underlying kernel</a></div><div class="lev3 toc-item"><a href="#Exact-sampling-scheme" data-toc-modified-id="Exact-sampling-scheme-425"><span class="toc-item-num">4.2.5&nbsp;&nbsp;</span>Exact sampling scheme</a></div><div class="lev3 toc-item"><a href="#MCMC-sampling" data-toc-modified-id="MCMC-sampling-426"><span class="toc-item-num">4.2.6&nbsp;&nbsp;</span>MCMC sampling</a></div><div class="lev2 toc-item"><a href="#II.-3.-Exotic-DPPs" data-toc-modified-id="II.-3.-Exotic-DPPs-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>II. 3. Exotic DPPs</a></div><div class="lev3 toc-item"><a href="#See-documentation" data-toc-modified-id="See-documentation-431"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>See <a href="https://dppy.readthedocs.io/en/latest/exotic_dpps/index.html" target="_blank">documentation</a></a></div><div class="lev3 toc-item"><a href="#a.-Uniform-Spanning-Trees" data-toc-modified-id="a.-Uniform-Spanning-Trees-432"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>a. Uniform Spanning Trees</a></div><div class="lev4 toc-item"><a href="#Uniform-spanning-trees-of-a-connected-graph" data-toc-modified-id="Uniform-spanning-trees-of-a-connected-graph-4321"><span class="toc-item-num">4.3.2.1&nbsp;&nbsp;</span>Uniform spanning trees of a connected graph</a></div><div class="lev4 toc-item"><a href="#Display-kernel" data-toc-modified-id="Display-kernel-4322"><span class="toc-item-num">4.3.2.2&nbsp;&nbsp;</span>Display kernel</a></div><div class="lev4 toc-item"><a href="#Sample-a-UST" data-toc-modified-id="Sample-a-UST-4323"><span class="toc-item-num">4.3.2.3&nbsp;&nbsp;</span>Sample a UST</a></div><div class="lev3 toc-item"><a href="#b.-(Poissonized)-Plancherel" data-toc-modified-id="b.-(Poissonized)-Plancherel-433"><span class="toc-item-num">4.3.3&nbsp;&nbsp;</span>b. (Poissonized) Plancherel</a></div><div class="lev4 toc-item"><a href="#Choose-a-$\theta$-to-sample-a-permutation-$\sigma-\in-\mathfrak{S}_N$-with-$N-\sim-\mathcal{P}(\theta)$" data-toc-modified-id="Choose-a-$\theta$-to-sample-a-permutation-$\sigma-\in-\mathfrak{S}_N$-with-$N-\sim-\mathcal{P}(\theta)$-4331"><span class="toc-item-num">4.3.3.1&nbsp;&nbsp;</span>Choose a <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-44-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03B8;</mi></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-494" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.491em, 1000.48em, 2.503em, -999.997em); top: -2.318em; left: 0em;"><span class="mrow" id="MathJax-Span-495"><span class="mi" id="MathJax-Span-496" style="font-family: STIXMathJax_Normal-italic;">𝜃</span></span><span style="display: inline-block; width: 0px; height: 2.324em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>θ</mi></math></span></span><script type="math/tex" id="MathJax-Element-44">\theta</script> to sample a permutation <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-45-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>&amp;#x03C3;</mi><mo>&amp;#x2208;</mo><msub><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi mathvariant=&quot;fraktur&quot;>S</mi></mrow><mi>N</mi></msub></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-497" style="width: 4.11em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.396em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1003.4em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-498"><span class="mi" id="MathJax-Span-499" style="font-family: STIXMathJax_Normal-italic;">𝜎<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-500" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-501" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1000.78em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-502"><span class="mrow" id="MathJax-Span-503"><span class="mi" id="MathJax-Span-504" style="font-family: STIXMathJax_Fraktur;">𝔖</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.836em;"><span class="mi" id="MathJax-Span-505" style="font-size: 70.7%; font-family: STIXMathJax_Normal-italic;">𝑁<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>σ</mi><mo>∈</mo><msub><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="fraktur">S</mi></mrow><mi>N</mi></msub></math></span></span><script type="math/tex" id="MathJax-Element-45">\sigma \in \mathfrak{S}_N</script> with <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-46-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>N</mi><mo>&amp;#x223C;</mo><mrow class=&quot;MJX-TeXAtom-ORD&quot;><mi class=&quot;MJX-tex-caligraphic&quot; mathvariant=&quot;script&quot;>P</mi></mrow><mo stretchy=&quot;false&quot;>(</mo><mi>&amp;#x03B8;</mi><mo stretchy=&quot;false&quot;>)</mo></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-506" style="width: 4.884em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1003.99em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-507"><span class="mi" id="MathJax-Span-508" style="font-family: STIXMathJax_Normal-italic;">𝑁<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-509" style="font-family: STIXMathJax_Main; padding-left: 0.301em;">∼</span><span class="texatom" id="MathJax-Span-510" style="padding-left: 0.301em;"><span class="mrow" id="MathJax-Span-511"><span class="mi" id="MathJax-Span-512" style="font-family: STIXMathJax_Variants-italic;"><span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span class="mo" id="MathJax-Span-513" style="font-family: STIXMathJax_Main;">(</span><span class="mi" id="MathJax-Span-514" style="font-family: STIXMathJax_Normal-italic;">𝜃</span><span class="mo" id="MathJax-Span-515" style="font-family: STIXMathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>N</mi><mo>∼</mo><mrow class="MJX-TeXAtom-ORD"><mi class="MJX-tex-caligraphic" mathvariant="script">P</mi></mrow><mo stretchy="false">(</mo><mi>θ</mi><mo stretchy="false">)</mo></math></span></span><script type="math/tex" id="MathJax-Element-46">N \sim \mathcal{P}(\theta)</script></a></div><div class="lev3 toc-item"><a href="#c.-Carries-Process" data-toc-modified-id="c.-Carries-Process-434"><span class="toc-item-num">4.3.4&nbsp;&nbsp;</span>c. Carries Process</a></div><div class="lev4 toc-item"><a href="#Choose-base-$b$-to-sample-i.i.d.-digits-in-$\{0,-\dots,-b-1\}$" data-toc-modified-id="Choose-base-$b$-to-sample-i.i.d.-digits-in-$\{0,-\dots,-b-1\}$-4341"><span class="toc-item-num">4.3.4.1&nbsp;&nbsp;</span>Choose base <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-47-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mi>b</mi></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-516" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.491em, 1000.48em, 2.503em, -999.997em); top: -2.318em; left: 0em;"><span class="mrow" id="MathJax-Span-517"><span class="mi" id="MathJax-Span-518" style="font-family: STIXMathJax_Normal-italic;">𝑏</span></span><span style="display: inline-block; width: 0px; height: 2.324em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>b</mi></math></span></span><script type="math/tex" id="MathJax-Element-47">b</script> to sample i.i.d. digits in <span class="MathJax_Preview" style="color: inherit;"></span><span class="MathJax" id="MathJax-Element-48-Frame" tabindex="0" data-mathml="<math xmlns=&quot;http://www.w3.org/1998/Math/MathML&quot;><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>{</mo><mn>0</mn><mo>,</mo><mo>&amp;#x2026;</mo><mo>,</mo><mi>b</mi><mo>&amp;#x2212;</mo><mn>1</mn><mo fence=&quot;false&quot; stretchy=&quot;false&quot;>}</mo></math>" role="presentation" style="position: relative;"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-519" style="width: 6.729em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.67em, 1005.48em, 2.86em, -999.997em); top: -2.497em; left: 0em;"><span class="mrow" id="MathJax-Span-520"><span class="mo" id="MathJax-Span-521" style="font-family: STIXMathJax_Main;">{</span><span class="mn" id="MathJax-Span-522" style="font-family: STIXMathJax_Main;">0</span><span class="mo" id="MathJax-Span-523" style="font-family: STIXMathJax_Main;">,</span><span class="mo" id="MathJax-Span-524" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">…</span><span class="mo" id="MathJax-Span-525" style="font-family: STIXMathJax_Main; padding-left: 0.182em;">,</span><span class="mi" id="MathJax-Span-526" style="font-family: STIXMathJax_Normal-italic; padding-left: 0.182em;">𝑏</span><span class="mo" id="MathJax-Span-527" style="font-family: STIXMathJax_Main; padding-left: 0.241em;">−</span><span class="mn" id="MathJax-Span-528" style="font-family: STIXMathJax_Main; padding-left: 0.241em;">1</span><span class="mo" id="MathJax-Span-529" style="font-family: STIXMathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.503em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mo>…</mo><mo>,</mo><mi>b</mi><mo>−</mo><mn>1</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script type="math/tex" id="MathJax-Element-48">\{0, \dots, b-1\}</script></a></div><div class="lev1 toc-item"><a href="#III.-Tools-behind-the-scene" data-toc-modified-id="III.-Tools-behind-the-scene-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>III. Tools behind the scene</a></div><div class="lev1 toc-item"><a href="#IV.---Suggestions/needs?" data-toc-modified-id="IV.---Suggestions/needs?-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>IV.   Suggestions/needs?</a></div>

# [DPPy](https://github.com/guilgautier/DPPy) tutorial

**First used for tutorial at Groupe de Travail DPPs, 10/29/2018 - Painlevé, Univ. Lille 1**

**Why are we here ?**

**DPPy**
  + Acronym: DPP + Python
  + Python Toolbox for (sampling) DPPs
 
**Goal**
  + showcase the DPP samplers featured in DPPy
  + present tools behind the scene 
  
  [GitHub](https://github.com/guilgautier/DPPy)
  [![Documentation Status](https://readthedocs.org/projects/dppy/badge/?version=latest)](https://dppy.readthedocs.io/en/latest/?badge=latest)
  [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)
  + needs/suggestion for improvement

# I. Check installation
---

[Instructions](https://github.com/guilgautier/DPPy#installation)

**Any issue?**

- [Travis](https://travis-ci.com/guilgautier/DPPy) says **no** :-)
    [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)




# 💣 To interact with source code, please uncomment the following cell

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, os.path.abspath('..'))

# II. Sample DPPs with DPPy
----

**Main DPPy objects' methods**

**`.sample()`**
**`.plot()`**

## II. 1. Continuous DPPs: $\beta$-Ensembles

### See [documentation](https://dppy.readthedocs.io/en/latest/continuous_dpps/index.html)

$$
(x_1,\dots,x_N) 
	\sim 
		\frac{1}{Z_{N,\beta}}
		\left|\Delta(x_1,\dots,x_N)\right|^{\beta}
		\prod_{i= 1}^N 
			\mu(d x_i)
$$

- Random matrix models
- $\beta=2$: projection DPPs

### d. Circular

In [ ]:
from dppy.beta_ensembles import CircularEnsemble

circular = CircularEnsemble(beta=2)  # beta must be >=0 integer, default beta=2

#### Visualization of the cristallization as $\beta\in \mathbb{N}^*$ increases 

In [ ]:
#@title ##### Use a slider! (on G-Colab only!!)

_beta = 2  #@param {type:'slider', min:0, max:100, step:1}
_size = 30  #@param {type:'slider', min:0, max:100, step:1}

circular.beta = _beta
circular.sample_banded_model(size_N=_size)
circular.plot()

##### Or simply a loop

In [ ]:
for b in (0, 1, 5, 10):
    circular.beta = b
    circular.sample_banded_model(size_N=30)
    circular.plot()

#### Sample using full matrix model, $\beta \in \{0, 1, 2, 4\}$

In [ ]:
circular.beta = 2
# 1. Plot the eigenvalues, they lie on the unit circle
circular.sample_full_model(size_N=30, haar_mode='Hermite')  # Sample # haar_mode = 'Hermite'/'QR'
circular.plot() # Plot of the eigenvalues

# 2. Histogram of the angle of more points, should look uniform on [0,2pi]
circular.sample_full_model(size_N=1000, haar_mode='Hermite')  # Sample
circular.hist()

#### Sample using banded (quindiagonal) matrix model, $\beta \in \mathbb{N}^*$

In [ ]:
circular.beta = 8
circular.sample_banded_model(size_N=30)
circular.plot()

circular.sample_banded_model(size_N=1000)
circular.hist()

##### $\beta=0$, i.i.d. uniform points on the unit circle

In [ ]:
circular.beta = 0
circular.sample_banded_model(size_N=30)
circular.plot()

### a. Hermite $\mu = \mathcal{N}$

In [ ]:
from dppy.beta_ensembles import HermiteEnsemble

hermite = HermiteEnsemble(beta=4)  # beta can be >=0, default beta=2

#### Sample using full matrix model, $\beta \in \{0, 1, 2, 4\}$

In [ ]:
hermite.sample_full_model(size_N=500)
# hermite.plot(normalization=True)
hermite.hist(normalization=True)

###### To compare with the sampling speed of the tridiagonal model simply use

In [ ]:
hermite.sample_banded_model(size_N=500)
hermite.hist(normalization=True)

#### Sample using banded (tridiagonal) matrix model $\beta \in \mathbb{R}_+$ 

In [ ]:
# beta can be >= 0, default beta=2
hermite.beta= 5.43  # Or hermite = HermiteEnsemble(beta=5.43)
# Reference measure is N(mu, sigma^2)
hermite.sample_banded_model(loc=0.0, scale=1.0, size_N=500)
# hermite.plot(normalization=True)
hermite.hist(normalization=True)

##### $\beta=0$, i.i.d. Gaussian

In [ ]:
# beta can be >= 0, default beta=2
hermite.beta= 0  # Or hermite = HermiteEnsemble(beta=5.43)
# Reference measure is N(mu, sigma^2)
hermite.sample_banded_model(size_N=1000)
# hermite.plot(normalization=True)
hermite.hist(normalization=True)  # True: N(0,2) as in full matrix model


### b. Laguerre $\mu = \mathcal{\Gamma}$

In [ ]:
from dppy.beta_ensembles import LaguerreEnsemble

laguerre = LaguerreEnsemble(beta=1)  # beta can be >= 0, default beta=2

#### Sample using full matrix model, $\beta \in \{0, 1, 2, 4\}$

In [ ]:
laguerre.sample_full_model(size_N=500, size_M=800)  # M >= N
# laguerre.plot(normalization=True)
laguerre.hist(normalization=True)

###### To compare with the sampling speed of the tridiagonal model simply use

In [ ]:
laguerre.sample_banded_model(size_N=500, size_M=800)
laguerre.hist(normalization=True)

#### Sample using banded (tridiagonal) matrix model, $\beta \in \mathbb{R}_+$

In [ ]:
laguerre. beta = 2.98  # Or laguerre = LaguerreEnsemble(beta=2.98)
# Reference measure is Gamma(k, theta)
laguerre.sample_banded_model(shape=600, scale=2.0, size_N=400)
# laguerre.plot(normalization=True)
laguerre.hist(normalization=True)

##### $\beta=0$, i.i.d. $\Gamma(k,\theta)$
Error if using `size_M` as in full matrix model since `shape= k = 0`

In [ ]:
laguerre.beta = 0 
# Reference measure is Gamma(k, theta)
laguerre.sample_banded_model(shape=6, scale=4.0, size_N=1000)
# laguerre.plot(normalization=True)
laguerre.hist(normalization=True)  # True: Gamma(shape, 2) as in full matrix model

### c. Jacobi $\mu = \operatorname{Beta}$

In [ ]:
from dppy.beta_ensembles import JacobiEnsemble

jacobi = JacobiEnsemble(beta=2)  # beta can be >= 0, default beta=2

#### Sample using full matrix model,  $\beta \in \{0, 1, 2, 4\}$

In [ ]:
jacobi.sample_full_model(size_N=400, size_M1=500, size_M2=600)  # M_1, M_2 >= N
# jacobi.plot(normalization=True)
jacobi.hist(normalization=True)

###### To compare with the sampling speed of the tridiagonal model simply use

In [ ]:
jacobi.sample_banded_model(size_N=400, size_M1=500, size_M2=600)
jacobi.hist(normalization=True)

#### Sample using banded (tridiagonal) matrix model,  $\beta \in \mathbb{R}_+$

In [ ]:
# beta can be >= 0, default beta=2
jacobi.beta = 3.14  # Or jacobi = JacobiEnsemble(beta=3.14) 
# Reference measure is Beta(a,b)
jacobi.sample_banded_model(a=500, b=300, size_N=400)
# jacobi.plot(normalization=True)
jacobi.hist(normalization=True)

##### $\beta=0$, i.i.d. $\operatorname{Beta}(a,b)$
Error if using `size_M1,2` as in full matrix model since `a,b = 0`

In [ ]:
jacobi.beta = 0 
# Reference measure is Beta(a, b)
jacobi.sample_banded_model(a=6, b=4, size_N=1000)
# jacobi.plot(normalization=True)
jacobi.hist(normalization=True)  # True/False: Beta(a, b)

### (e) Ginibre $\beta = 2$ only

In [ ]:
from dppy.beta_ensembles import GinibreEnsemble

ginibre = GinibreEnsemble()  # beta must be 2 (default)

In [ ]:
ginibre.sample_full_model(size_N=40)
ginibre.plot(normalization=True)

## II. 2. Discrete DPPs

### See [documentation](https://dppy.readthedocs.io/en/latest/finite_dpps/index.html)

In [ ]:
from numpy.random import rand, randn
from scipy.linalg import qr

from dppy.finite_dpps import FiniteDPP

### Build inclusion kernel $\mathbf{K}$

$\operatorname{DPP}(\mathbf{K})$, with $\mathbf{K}\in\mathbb{R}^{N\times N}$ 

$$
	\mathbf{K} = \sum_{n=1}^{N} \lambda_n u_n u_n^{\top}
$$

In [ ]:
r, N = 10, 10

# Random orthogonal vectors
eig_vecs, _ = qr(randn(N, r), mode='economic')
# Random eigenvalues
eig_vals = rand(r)  # 0< <1
#eig_vals = np.random.randint(2, size=r) # 0 or 1 i.e. projection

K = eig_vecs*eig_vals.dot(eig_vecs.T)

### Declare a finite DPP in DPPy

#### 1. Via eigen-decomposition

In [ ]:
DPP = FiniteDPP('inclusion', **{'K_eig_dec': (eig_vals, eig_vecs)})

#print(DPP.K)

#### 2. Via its kernel

In [ ]:
DPP = FiniteDPP('inclusion', **{'K': K})

### Other features


#### a. Compute the *other* kernel 
- $L=K(I-K)^{-1}$

In [ ]:
DPP = FiniteDPP('inclusion', **{'K_eig_dec': (eig_vals, eig_vecs)})
print(DPP.L)
DPP.compute_L()

- $K=L(I+L)^{-1}$

In [ ]:
eig_vals = 4*rand(r)  # >=0
DPP = FiniteDPP('marginal', **{'L_eig_dec': (eig_vals, eig_vecs)})
print(DPP.L)
DPP.compute_K()

#### b. Compute/plot the underlying kernel


In [ ]:
eig_vals = rand(r)  # 0< <1
DPP = FiniteDPP('inclusion', **{'K_eig_dec': (eig_vals, eig_vecs)})
DPP.plot_kernel()

### Exact sampling scheme

- $\operatorname{DPP}(\mathbf{K})$, with $\mathbf{K}\in\mathbb{R}^{N\times N}$ 

$$
	\mathbf{K} = \sum_{n=1}^{N} \lambda_n u_n u_n^{\top}
$$

1. Draw independent $\operatorname{\mathcal{B}er}(\lambda_n)$ for each eigenvector $u_n$ and store the selected ones in $\tilde{U}$.
2. Sample from the corresponding *projection* $\operatorname{DPP}(\tilde{U}\tilde{U}^{\top})$.

In [ ]:
# Sample
for _ in range(10):
    DPP.sample_exact()

DPP.list_of_samples

In [ ]:
DPP.flush_samples()
DPP.list_of_samples

- $\operatorname{k-DPP}(\mathbf{L})$

$$\mathbb{P}[\mathcal{X} = S] \propto \det \mathbf{L}_S ~~ 1_{|S|=k}$$

In [ ]:
r, N = 5, 10

# Random feature vectors
Phi = randn(r, N)
L = Phi.T.dot(Phi)
DPP = FiniteDPP('marginal', **{'L': L})

k = 3
for _ in range(10):
    DPP.sample_exact_k_dpp(size=3)

DPP.list_of_samples

### MCMC sampling

At state $S\subset [N]$, propose $S'$ different from $S$ by at most 2 elements by picking

**1. Local moves**
- $s \sim \mathcal{U}_{S},  t \sim \mathcal{U}_{[N]\setminus S}$
- $|S'\Delta S| \leq 1$

In [ ]:
r, N = 4, 10
Phi = randn(r, N)
L = Phi.T.dot(Phi)
DPP = FiniteDPP('marginal', **{'L': L})

##### a. Exchange: 
$$S' \leftrightarrow S \setminus s \cup t$$

In [ ]:
DPP.flush_samples()
DPP.sample_mcmc('E')  #AD, ADE
print(DPP.list_of_samples)

##### b. Add-Delete:
  - Add $S' \leftrightarrow S \cup t$
  - Delete $S' \leftrightarrow S \setminus s$

In [ ]:
DPP.flush_samples()
DPP.sample_mcmc('AD')  #E, AD
print(DPP.list_of_samples)

In [ ]:
eig_vecs, _ = qr(randn(N, r), mode='economic')
eig_vals = rand(r)

DPP = FiniteDPP('inclusion', **{'K_eig_dec': (eig_vals, eig_vecs)})
DPP.sample_mcmc('AD')
print(DPP.list_of_samples)

##### c. ADE

In [ ]:
DPP.flush_samples()
DPP.sample_mcmc('AED')  #E, AD
print(DPP.list_of_samples)

**2. Zonotope**

In [ ]:
r, N = 4, 10
A = randn(r, N)

DPP = FiniteDPP('inclusion', projection=True, **{'A_zono': A})

DPP.sample_mcmc('zonotope')
print(DPP.list_of_samples)

**3. k-DPP**

To preserve the cardinality of the samples, only exchange moves can be performed

In [ ]:
r, N = 5, 10

# Random feature vectors
Phi = randn(r, N)
L = Phi.T.dot(Phi)
DPP = FiniteDPP("marginal", **{"L":L})

k = 3
DPP.sample_mcmc_k_dpp(size=k)
print(DPP.list_of_samples)

## II. 3. Exotic DPPs
### See [documentation](https://dppy.readthedocs.io/en/latest/exotic_dpps/index.html)

- Uniform spanning trees
- Plancherel
- Descent Processes

### a. Uniform Spanning Trees

#### Uniform spanning trees of a connected graph

In [ ]:
import networkx as nx

from dppy.exotic_dpps import UST

In [ ]:
g = nx.Graph()
edges = [(0,2), (0,3), (1,2), (1,4), (2,3), (2,4), (3,4)]
g.add_edges_from(edges)

ust = UST(g)

ust.plot_graph()

#### Display kernel

In [ ]:
ust.compute_kernel()
ust.plot_kernel()

#### Sample a UST

In [ ]:
for md in ('Aldous-Broder', 'Wilson', 'DPP_exact'):
    ust.sample(md)
    ust.plot()

### b. (Poissonized) Plancherel

#### Choose a $\theta$ to sample a permutation $\sigma \in \mathfrak{S}_N$ with $N \sim \mathcal{P}(\theta)$

In [ ]:
from dppy.exotic_dpps import PoissonizedPlancherel

In [ ]:
theta = 1500  # Poisson parameter
pp_dpp = PoissonizedPlancherel(theta=theta)
pp_dpp.sample()
pp_dpp.plot_diagram(True)

### c. Carries Process

#### Choose base $b$ to sample i.i.d. digits in $\{0, \dots, b-1\}$

In [ ]:
from dppy.exotic_dpps import CarriesProcess

In [ ]:
#@title ##### Use a slider!

_base = 10  #@param {type:'slider', min:0, max:10, step:1}
_size = 63  #@param {type:'slider', min:0, max:1000, step:1}

cp = CarriesProcess(_base)

cp.sample(_size)

cp.plot_vs_bernoullis()

# III. Tools behind the scene

---


 - Host collaborative project [GitHub](https://github.com/guilgautier/DPPy)
 - Documentation [![Documentation Status](https://readthedocs.org/projects/dppy/badge/?version=latest)](https://dppy.readthedocs.io/en/latest/?badge=latest)
 - Continuous integration [![Build Status](https://travis-ci.com/guilgautier/DPPy.svg?branch=master)](https://travis-ci.com/guilgautier/DPPy)
 
 
**Reproducible reasearch**

 - [DPPy](https://github.com/guilgautier/DPPy)
 - companion paper [DPPy_paper](https://github.com/guilgautier/DPPy_paper)

# IV.   Suggestions/needs?

---

**[Raise an issue on GitHub](https://github.com/guilgautier/DPPy/issues) :)**

*   
*
*
*
*
*   
*
*
*
*